In [ ]:
#답이미지가 존재하지 않는 잘린 페이지이어 붙이기 

In [ ]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install Pillow
! pip install pytesseract
! sudo apt-get install tesseract-ocr-script-hang tesseract-ocr-script-hang-vert

In [ ]:
#라이브러리 임포트
import numpy as np
import cv2 #openCV package
from google.colab.patches import cv2_imshow
import re
import pytesseract
import glob
import os
import time
from PIL import ImageEnhance, ImageFilter, Image
#from pytesseract import Output
from matplotlib import pyplot as plt


#구글 드라이브와 연동 
from google.colab import drive
drive.mount('/gdrive')


In [ ]:

#함수 이름을 설정하고 저장하는 함수 
def include(cropped,num):

    global qnum
    number = str(num)
    #
    if (len(number)==1):
        number = '000'+str(num)
    elif (len(number)==2):
        number = '00'+str(num)
    else:
        number = '0'+str(num)


    newfile=save_path+str(num)+"_pg"+j+".png"
    #print(newfile)
    #print(num)
    #cv2_imshow(cropped)
    #print("===============================================")
    cv2.imwrite(newfile,cropped)
    qnum=qnum+1
    

#수능완성 문제집의 경로지정코드 
def make_path_SW():

    book_list=['2021','2020','2019','2018','2017']
    for i in range(len(book_list)):
        path=book_list[i]+'/'
        path_list.append(path)



#flag를 기준으로 이미지 합치기 
def combine_image(image_url):
    global flag
    global image_arr
    global noans_count
    #image_arr=[]

    image = cv2.imread(image_url)
    imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 

    template = cv2.imread('/gdrive/MyDrive/ProjectStudy/answer_temp.png',0)


    res = cv2.matchTemplate(imgray, template, cv2.TM_CCOEFF_NORMED)
    threshold = .55
    loc = np.where(res >= threshold)

    length = len(loc[0])

    #답이 없음
    if (length <= 0):
        image_arr[noans_count] = image_url
        noans_count=noans_count+1
        flag = 0
    elif (length >0 and flag ==0):

        for i in range(noans_count-1,-1,-1):
            image_noans = cv2.imread(image_arr[i])
            image = cv2.vconcat([image_noans,image])
            noans_count = noans_count-1
        include(image,qnum)
        if (noans_count < 0):
            noans_count = 0
        flag = 1
    else: #있음(그냥 저장)
        include(image,qnum)
        flag=1   




#함수실행 
if __name__ == "__main__":
    path_list=[]
    make_path_SW()
    path_prev='/gdrive/MyDrive/ProjectStudy/answers/'
    path_back='/Suwan_na/'

    for i in range(len(path_list)):
        qnum=1
        flag = 1
        noans_count=0
        image_arr = [None]*8

        load_path=path_prev+'revised_cropped'+path_back+path_list[i]
        save_path=path_prev+'cropped_final'+path_back+path_list[i]
        #template_path = path_prev+'template/'+path_list[i]+'template.png'
        os.chdir(load_path)
    
        images=glob.glob('./*.png')
        #폴더내 이미지 순서대로 불러오기
        images.sort()
        #print(images)
       
        for i in range(len(images)):
            filename=images[i]
            page_num=filename[-7:-4]
            
            page_url = load_path+images[i]
            j=page_num
            #print(j)
            #print(page_url)
            combine_image(page_url)
            #problem_crop(filename,j)



